In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

In [4]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 150

In [5]:
graph = nx.complete_graph(100)

In [ ]:
MC_STEPS = 1000
Q = 4
N = 100

In [6]:
def get_neighbors(graph):
    d = {}
    for node in list(graph.nodes()):
        d[node] = list(graph.neighbors(node))
    return d

In [ ]:
def get_opinions(opinion1_prop, nodes_num):
    opinion1 = int(opinion1_prop*nodes_num)
    opinions = [-1]*(nodes_num - opinion1) + [1]*opinion1
    random.shuffle(opinions)
    opinions_dict = {i:opinions[i] for i in range(nodes_num)}
    return opinions_dict

In [ ]:
def q_voter_model(graph, p, q, epsilon, opinion1, MC_steps):
    opinions = get_opinions(opinion1, nx.number_of_nodes(graph))
    neighbors = get_neighbors(graph)
    nodes = list(neighbors.keys())
    n = len(nodes)
    magn = np.zeros(MC_steps)
    for i in range(MC_steps):
        nodes = np.random.randint(0, n, size=n)
        ps = np.random.uniform(size=n) < p
        epsilons = np.random.uniform(size=n) < epsilon
        
        for j, node in enumerate(nodes):
            node_opinion = opinions[node]
            contrary = -1 if node_opinion == 1 else 1
            if ps[j]:
                opinions[node] = contrary if random.uniform(0, 1) < 0.5 else node_opinion
            else:
                curr_q = q
                if q > len(neighbors[node]):
                    curr_q = len(neighbors[node])
                neighbor_list = random.choices(neighbors[node], k=curr_q)
                neighbors_opinions = [opinions[i] for i in neighbor_list]
                if neighbors_opinions.count(node_opinion) == 0:
                    opinions[node] = contrary
                elif neighbors_opinions.count(contrary) != 0 and epsilons[j]:
                    opinions[node] = contrary
        magn[i] = sum(opinions.values())/n
    return opinions, magn

In [ ]:
def q_voter_model_new(graph, p, q, f, epsilon, opinion1, MC_steps):
    # adjust
    opinions = get_opinions(opinion1, nx.number_of_nodes(graph))
    neighbors = get_neighbors(graph)
    nodes = list(neighbors.keys())
    n = len(nodes)
    magn = np.zeros(MC_steps)
    for i in range(MC_steps):
        nodes = np.random.randint(0, n, size=n)
        ps = np.random.uniform(size=n) < p
        #epsilons = np.random.uniform(size=n) < epsilon
        
        for j, node in enumerate(nodes):
            node_opinion = opinions[node]
            contrary = -1 if node_opinion == 1 else 1
            if ps[j]:
                opinions[node] = contrary if random.uniform(0, 1) < f else node_opinion
            else:
                curr_q = q
                if q > len(neighbors[node]):
                    curr_q = len(neighbors[node])
                neighbor_list = random.choices(neighbors[node], k=curr_q)
                neighbors_opinions = [opinions[i] for i in neighbor_list]
                if neighbors_opinions.count(node_opinion) == 0:
                    opinions[node] = contrary
                # elif neighbors_opinions.count(contrary) != 0 and epsilons[j]:
                #     opinions[node] = contrary
        magn[i] = sum(opinions.values())/n
    return opinions, magn

In [ ]:
ps = np.arange(0, 1, 0.05)

In [7]:
fs = [0.2, 0.3, 0.4, 0.5]

In [ ]:
def simulate_q_voter_CG(p, q, epsilon, MC_steps=1000, steps=100, opinion1=1, N=100):
    graph = nx.complete_graph(N)
    results = np.zeros((steps, MC_steps))
    for i in range(steps):
        _, magn = q_voter_model(graph, p, q, epsilon, opinion1, MC_steps)
        results[i] = abs(magn)
    return np.mean(results, axis = 0)

## Magnetization

In [ ]:
def simulate_magnetization(ps, f, Q, MC_steps=1000, steps=100, graph_size=100):
    final_magn = np.zeros(len(ps))
    for i, p in enumerate(ps):
        print(i)
        final_magn[i] = simulate_q_voter_CG(p, Q, f, MC_steps=MC_steps, steps=steps, opinion1=1, N=graph_size)[-1]

In [8]:
MC_STEPS = 1000
repetitions = 100
size=100

In [ ]:
magn = list(np.zeros(len(fs)))
for i, f in enumerate(fs):
    magn[i] = simulate_magnetization(ps, f, Q=4, MC_steps=MC_STEPS, steps=repetitions, graph_size=100)